In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys, os, copy, time

import numpy as np
from gpt import GPT
from GPT_tools.GPTExtension import run_gpt_with_particlegroup
from GPT_tools.cathode_particlegroup import get_cathode_particlegroup
from GPT_tools.gpt_plot import gpt_plot, gpt_plot_dist1d, gpt_plot_dist2d, gpt_plot_trajectory
from GPT_tools.gpt_plot_gui import gpt_plot_gui
from GPT_tools.tools import get_screen_data

In [ ]:
template_dir = '.'
DISTGEN_INPUT_FILE = os.path.join(template_dir,'distgen.in.yaml')
GPT_INPUT_FILE = os.path.join(template_dir,'gpt.in')

# Machine settings

In [ ]:
settings = {}

settings['gun_voltage'] = 140 
settings['buncher_voltage'] = 3.47
settings['buncher_phi_rel'] = -90.0

settings['sol_1_current'] = 0.0
settings['sol_2_current'] = 1.995

settings['total_charge:value'] = 5
settings['total_charge:units'] = 'fC'

settings['start:MTE:value'] = 80
settings['start:MTE:units'] = 'meV'

settings['transforms:t0:sigma_xy:value'] = 18.0
settings['transforms:t0:sigma_xy:units'] = 'um'

settings['transforms:t1:sigma_t:value'] = 8.4
settings['transforms:t1:sigma_t:units'] = 'ps'
settings['t_dist:alpha:value'] = 0.25

In [ ]:
settings['space_charge'] = 1
settings['GBacc'] = 5.5

settings['n_particle'] = 2000
settings['n_screens'] = 100

# Get and plot initial distribution

In [ ]:
PG = get_cathode_particlegroup(settings, DISTGEN_INPUT_FILE, verbose=False)

In [ ]:
gpt_plot_dist2d(PG, 'x', 'y', axis='equal')

In [ ]:
gpt_plot_dist1d(PG, 't')

# Run GPT

In [ ]:
gpt_data = run_gpt_with_particlegroup(settings,
                         gpt_input_file=GPT_INPUT_FILE,
                         input_particle_group=PG,
                         verbose=True,
                         gpt_verbose=False,
                         auto_phase=True,
                         timeout=10000)

# Plot GUI

In [ ]:
gpt_plot_gui(gpt_data)

# Individual example plots

In [ ]:
# Example trend plot
# ----------------------------------------------------------------------------------------------------------------------------------------------------------
# Syntax: gpt_plot(gpt_data, var_1, var_2)     , variables can be e.g. 'mean_z', 'sigma_t', 'norm_emit_x', etc.

# Options include:
#      units = y_units            , units to use on the y-axis
#      xlim = [min, max]          , x axis range
#      ylim = [min, max]          , y axis range
#      color = list               , colors to use for each line
#      log_scale = True           , Plot y-axis with log scale
#      include_ids = list         , show only particles with listed ids
#      show_survivors_at_z = z    , show only particles that have survived to the screen at z, postprocessing options (see below) can be applied to this screen
#      show_cursor = True         , show a data tooltip when you mouse over the line
#      show_screens = True        , show so-called "special" screens with individual points in the plot

gpt_plot(gpt_data, 'mean_z', ['sigma_x', 'sigma_y'], color = ['red', 'green'])

In [ ]:
# Example 1D histogram
# ----------------------------------------------------------------------------------------------------------------------------------------------------------
# Syntax: gpt_plot_dist1d(gpt_data, var)     , variables can be 'x', 'y', 't', 'r', 'px', etc.

# Options include:
#      screen_z = z           , Get screen data from specific z value
#      tout_t = t             , Get tout data from specific t value
#      nbins = n              , Number of histogram bins
#      table_on = True        , include a table with summary statistics
#      plot_type = var_2      , what to plot on the y-axis, can be 'charge' (default) , 'sigma_x', 'norm_emit_x', etc.
#      xlim = [min, max]      , x axis range
#      ylim = [min, max]      , y axis range

gpt_plot_dist1d(gpt_data, 't', screen_z=1.843, nbins=30)

In [ ]:
# Example 2D histogram
# ----------------------------------------------------------------------------------------------------------------------------------------------------------
# Syntax: gpt_plot_dist2d(gpt_data, var_1, var_2)     , variables can be 'x', 'y', 't', 'r', 'px', etc.

# Options include:
#      screen_z = z           , Get screen data from specific z value
#      tout_t = t             , Get tout data from specific t value
#      nbins = [N, M]         , Number of histogram bins
#      table_on = True        , include a table with summary statistics
#      plot_type = type       , whether to show each particle, or a histogram. type can be either 'scatter' or 'histogram' (default)

gpt_plot_dist2d(gpt_data, 'x', 'px', screen_z = 1.843, nbins = [30, 30])

In [ ]:
# Example of postprocessing (used in any of the above plots as additional options)
# ----------------------------------------------------------------------------------------------------------------------------------------------------------
# Possible postprocessing options:
#           take_range = (variable, range_min, range_max))    , e.g. take_range=('r', 0, 1.0e-3) , show only particles within range of a parameter
#           take_slice = (variable, slice_index, n_slices))   , e.g. take_slice=('t', 24, 50)    , show only particles within the Nth slice of a parameter
#           clip_to_charge = final_charge                     , e.g. clip_to_charge=1.0e-12      , radially clip particles down to a given charge
#           clip_to_emit = final_emit                         , e.g. clip_to_emit=1.0e-6         , radially clip particles down to a given emittance
#           cylindrical_copies = n_copies                     , e.g. cylindrical_copies=50       , add N copies of each particle to enforce cylindrical symmetry
#           remove_spinning = True                            , e.g. remove_spinning = True      , removes correlations associated with being in a solenoid
#           remove_correlation = (variable_1, variable_2, N)  , e.g. remove_correlation = ('t', 'pz', 1)  , removes correlations between variables up to polynomial order N   

gpt_plot_dist2d(example_screen, 'x', 'px', cylindrical_copies=100)

In [ ]:
# Particle trajectories
# ----------------------------------------------------------------------------------------------------------------------------------------------------------
# Options include:
#      nlines = N                 , pick a random N lines to plot
#      show_survivors_at_z = z    , show only particles that have survived to the screen at z, postprocessing options (see above) can be applied to this screen
#      include_ids = list         , show only particles with listed ids
#      
gpt_plot_trajectory(gpt_data, 'z', 'r', nlines=200)

In [ ]:
# Get screen from z position
# ----------------------------------------------------------------------------------------------------------------------------------------------------------
z = 1.843
example_screen = get_screen_data(gpt_data, screen_z=z)[0]

# Save and Load

In [ ]:
save_directory = '.'
os.listdir(save_directory)

In [ ]:
file_to_save = 'example_save.h5'
gpt_data.archive(h5=os.path.join(save_directory, file_to_save))

In [ ]:
file_to_load = 'example_save.h5'
gpt_data = GPT()
gpt_data.load_archive(os.path.join(save_directory, file_to_load))